GLIE: Greedy in the Limit with Infinite Exploration

In [1]:
from rl.monte_carlo import *

from rl.function_approx import Tabular
from rl.distribution import Choose
from rl.chapter3.simple_inventory_mdp_cap import InventoryState
from rl.chapter10.prediction_utils import *

from rl.chapter3.simple_inventory_mdp_cap import SimpleInventoryMDPCap
from rl.dynamic_programming import value_iteration_result

from rl.distribution import Constant
from rl.dynamic_programming import V
import itertools
import rl.iterate as iterate
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.policy import FiniteDeterministicPolicy

In [2]:
# Setup Inventory MDP
capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0
gamma: float = 0.9
si_mdp: SimpleInventoryMDPCap = SimpleInventoryMDPCap(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)

In [3]:
# Get Value Function and Optimal Policy via Dynamic Programming
true_opt_vf, true_opt_policy = value_iteration_result(si_mdp, gamma=gamma)
print("True Optimal Value Function")
pprint(true_opt_vf)
print("True Optimal Policy")
print(true_opt_policy)

True Optimal Value Function
{NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.66095964467877,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.99189950444479,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.66095964467877,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.99189950444479,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855194671294,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991899504444792}
True Optimal Policy
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



# Question 1

## Implementing MC Control with GLIE

In [4]:
# Definitions

episode_length_tolerance: float = 1e-5
# GLIE:
epsilon_as_func_of_episodes: Callable[[int], float] = lambda k: k ** -1

initial_learning_rate: float = 0.1
half_life: float = 10000.0
exponent: float = 1.0

# Uniform sampling across state space:
initial_qvf_dict = {
    (s, a): 0. for s in si_mdp.non_terminal_states for a in si_mdp.actions(s)
}

# Function to control learning rate
learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)

# Redefine Tabular for QValue Function Approximation
QTabular = Tabular[Tuple[NonTerminal[S],A]]

### Tabular MC Control

In [5]:
# Tabular Monte Carlo Control:
def tabular_glie_mc_control(
    mdp: MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QTabular[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    episode_length_tolerance: float = 1e-6
) -> Iterator[QTabular[S, A]]:
    
    q: QTabular[S, A] = approx_0
    p: Policy[S, A] = epsilon_greedy_policy(q, mdp, 1.0) # Start off with epsilon = 1/1 = 1
    yield q

    num_episodes: int = 0
    while True:
        trace: Iterable[TransitionStep[S, A]] = \
            mdp.simulate_actions(states, p)
        num_episodes += 1
        for step in returns(trace, γ, episode_length_tolerance):
            q = q.update([((step.state, step.action), step.return_)])
        p = epsilon_greedy_policy(q, mdp, ϵ_as_func_of_episodes(num_episodes))
        yield q

### Test Tabular MC Control

In [6]:
# Test Tabular MC Control:

from rl.chapter11.control_utils import get_vf_and_policy_from_qvf

qvfs = tabular_glie_mc_control(
    mdp=si_mdp,
    states=Choose(si_mdp.non_terminal_states),
    approx_0=QTabular(
        values_map=initial_qvf_dict,
        count_to_weight_func=learning_rate_func
    ),
    γ=gamma,
    ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
    episode_length_tolerance=episode_length_tolerance
)

num_episodes = 10000
final_qvf: QTabular[InventoryState, int] = \
    iterate.last(itertools.islice(qvfs, num_episodes))

opt_vf, opt_policy = get_vf_and_policy_from_qvf(
    mdp=si_mdp,
    qvf=final_qvf
)
print("GLIE MC Optimal Value Function with {num_episodes:d} episodes")
pprint(opt_vf)
print("GLIE MC Optimal Policy with {num_episodes:d} episodes")
print(opt_policy)

GLIE MC Optimal Value Function with {num_episodes:d} episodes
{NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.84635362127615,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.780169223503503,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -29.08649156365046,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.64102026775767,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.67519361088475,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.829569146901967}
GLIE MC Optimal Policy with {num_episodes:d} episodes
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



### General MC Control

In [7]:
# General Monte Carlo Control:
def glie_mc_control(
    mdp: MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    episode_length_tolerance: float = 1e-6
) -> Iterator[QValueFunctionApprox[S, A]]:
   
    q: QValueFunctionApprox[S, A] = approx_0
    p: Policy[S, A] = epsilon_greedy_policy(q, mdp, 1.0) # Start off with epsilon = 1/1 = 1
    yield q

    num_episodes: int = 0
    while True:
        trace: Iterable[TransitionStep[S, A]] = \
            mdp.simulate_actions(states, p)
        num_episodes += 1
        for step in returns(trace, γ, episode_length_tolerance):
            q = q.update([((step.state, step.action), step.return_)])
        p = epsilon_greedy_policy(q, mdp, ϵ_as_func_of_episodes(num_episodes))
        yield q

## Implement Asset_alloc_discrete test

In [37]:
from rl.chapter7.asset_alloc_discrete import *
from pprint import pprint

In [38]:
steps: int = 4
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_stdev: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(
    2 / 3 * base_alloc,
    4 / 3 * base_alloc,
    11
)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = \
    [
        lambda _: 1.,
        lambda w_x: w_x[0],
        lambda w_x: w_x[1],
        lambda w_x: w_x[1] * w_x[1]
    ]
dnn: DNNSpec = DNNSpec(
    neurons=[],
    bias=False,
    hidden_activation=lambda x: x,
    hidden_activation_deriv=lambda y: np.ones_like(y),
    output_activation=lambda x: - np.sign(a) * np.exp(-x),
    output_activation_deriv=lambda y: -y
)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_stdev)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
    risky_return_distributions=risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr
)

In [11]:
#Optimal Value Function via ADP

vf_ff: Sequence[Callable[[NonTerminal[float]], float]] = [lambda _: 1., lambda w: w.state]
it_vf: Iterator[Tuple[DNNApprox[NonTerminal[float]], DeterministicPolicy[float, float]]] = \
    aad.backward_induction_vf_and_pi(vf_ff)

print("Backward Induction: VF And Policy")
print("---------------------------------")
print()
for t, (v, p) in enumerate(it_vf):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = p.action_for(init_wealth)
    val: float = v(NonTerminal(init_wealth))
    print(f"Opt Risky Allocation = {opt_alloc:.2f}, Opt Val = {val:.3f}")
    print("Weights")
    for w in v.weights:
        print(w.weights)
    print()

Backward Induction: VF And Policy
---------------------------------

Time 0

Opt Risky Allocation = 1.40, Opt Val = -0.212
Weights
[[0.2415891  1.30852079]]

Time 1

Opt Risky Allocation = 1.40, Opt Val = -0.246
Weights
[[0.17782372 1.22649341]]

Time 2

Opt Risky Allocation = 1.50, Opt Val = -0.283
Weights
[[0.1179855  1.14591944]]

Time 3

Opt Risky Allocation = 2.00, Opt Val = -0.323
Weights
[[0.05796964 1.07090809]]



## Test General MC Control on AssetAllocDiscrete

In [39]:
list_of_qvfs = []
for t in range(0,4):

    qvfs = glie_mc_control(
        mdp = aad.get_mdp(t),
        states = aad.get_states_distribution(t),
        approx_0 = aad.get_qvf_func_approx(),
        γ=gamma,
        ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
        episode_length_tolerance=episode_length_tolerance
    )


    final_qvf: QValueFunctionApprox[InventoryState, int] = \
        iterate.last(itertools.islice(qvfs, num_episodes))
    
    list_of_qvfs.append(final_qvf)

/var/folders/fj/qb_tch8j6sdg7th8gw7q9w000000gn/T/ipykernel_8361/3507441780.py:33: RuntimeWarning: overflow encountered in exp
  output_activation=lambda x: - np.sign(a) * np.exp(-x),


In [40]:
from pprint import pprint

for t, q in enumerate(list_of_qvfs):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = max(
        ((q((NonTerminal(init_wealth), ac)), ac) for ac in alloc_choices),
        key=itemgetter(0)
    )[1]
    val: float = max(q((NonTerminal(init_wealth), ac))
                     for ac in alloc_choices)
    print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
    print("Optimal Weights below:")
    for wts in q.weights:
        pprint(wts.weights)
    print()

Time 0

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[4.58359517, 0.02284127, 4.99079256, 6.14078238]])

Time 1

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[5.40012389, 0.12037042, 5.8081429 , 4.82104217]])

Time 2

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[4.27911185, 0.01554997, 5.11430369, 6.03359601]])

Time 3

Opt Risky Allocation = 1.000, Opt Val = -0.322
Optimal Weights below:
array([[ 1.76253743,  1.2036841 , -0.57129627, -1.26263872]])



# Question 2

#### Q-Learning is just SARSA, but the first action comes from the behavior policy and the second action comes from the target policy

In [60]:
from rl.td import epsilon_greedy_action


# Q-Tabular
def tabular_glie_sarsa(
    mdp: MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QTabular[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QTabular[S, A]]:
    q: QTabular[S, A] = approx_0
    yield q
    num_episodes: int = 0
    while True:
        num_episodes += 1
        ϵ: float = ϵ_as_func_of_episodes(num_episodes)
        state: NonTerminal[S] = states.sample()
        action: A = epsilon_greedy_action(
            q=q,
            nt_state=state,
            actions=set(mdp.actions(state)),
            ϵ=ϵ
        )
        steps: int = 0
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, NonTerminal):
                next_action: A = epsilon_greedy_action(
                    q=q,
                    nt_state=next_state,
                    actions=set(mdp.actions(next_state)),
                    ϵ=ϵ
                )
                q = q.update([(
                    (state, action),
                    reward + γ * q((next_state, next_action))
                )])
                action = next_action
            else:
                q = q.update([((state, action), reward)])
            yield q
            steps += 1
            state = next_state


In [61]:
# General Value Function

def glie_sarsa(
    mdp: MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q
    num_episodes: int = 0
    while True:
        num_episodes += 1
        ϵ: float = ϵ_as_func_of_episodes(num_episodes)
        state: NonTerminal[S] = states.sample()
        action: A = epsilon_greedy_action(
            q=q,
            nt_state=state,
            actions=set(mdp.actions(state)),
            ϵ=ϵ
        )
        steps: int = 0
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, NonTerminal):
                next_action: A = epsilon_greedy_action(
                    q=q,
                    nt_state=next_state,
                    actions=set(mdp.actions(next_state)),
                    ϵ=ϵ
                )
                q = q.update([(
                    (state, action),
                    reward + γ * q((next_state, next_action))
                )])
                action = next_action
            else:
                q = q.update([((state, action), reward)])
            yield q
            steps += 1
            state = next_state

In [62]:
max_episode_length: int = 100
epsilon_as_func_of_episodes: Callable[[int], float] = lambda k: k ** -0.5
initial_learning_rate: float = 0.1
half_life: float = 10000.0
exponent: float = 1.0
gamma: float = 0.9
initial_qvf_dict: Mapping[Tuple[NonTerminal[InventoryState], int], float] = {
    (s, a): 0. for s in si_mdp.non_terminal_states for a in si_mdp.actions(s)
}
learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)
qvfs: Iterator[QValueFunctionApprox[InventoryState, int]] = tabular_glie_sarsa(
    mdp=si_mdp,
    states=Choose(si_mdp.non_terminal_states),
    approx_0=Tabular(
        values_map=initial_qvf_dict,
        count_to_weight_func=learning_rate_func
    ),
    γ=gamma,
    ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
    max_episode_length=max_episode_length
)

In [63]:
num_updates = num_episodes * max_episode_length
final_qvf: QValueFunctionApprox[InventoryState, int] = \
    iterate.last(itertools.islice(qvfs, num_updates))
opt_vf, opt_policy = get_vf_and_policy_from_qvf(
    mdp=si_mdp,
    qvf=final_qvf
)
print(f"GLIE SARSA Optimal Value Function with {num_updates:d} updates")
pprint(opt_vf)
print(f"GLIE SARSA Optimal Policy with {num_updates:d} updates")
print(opt_policy)

GLIE SARSA Optimal Value Function with 1000000 updates
{NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.668996755388687,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.95015260489432,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.704929438008165,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.242889769065226,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.86617221756493,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.699963352671702}
GLIE SARSA Optimal Policy with 1000000 updates
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



### Compare to dynamic programming:

In [64]:
# Get Value Function and Optimal Policy via Dynamic Programming
true_opt_vf, true_opt_policy = value_iteration_result(si_mdp, gamma=gamma)
print("True Optimal Value Function")
pprint(true_opt_vf)
print("True Optimal Policy")
print(true_opt_policy)

True Optimal Value Function
{NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.99189950444479,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.99189950444479,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.66095964467877,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991899504444792,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855194671294,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.66095964467877}
True Optimal Policy
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



## Results between Tabular SARSA and dynamic programming are very similar

### Test on AssetAllocDiscrete

In [67]:
list_of_qvfs = []
for t in range(0,4):

    qvfs = glie_sarsa(
        mdp = aad.get_mdp(t),
        states = aad.get_states_distribution(t),
        approx_0 = aad.get_qvf_func_approx(),
        γ=gamma,
        ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
        max_episode_length=100
    )


    final_qvf: QValueFunctionApprox[InventoryState, int] = \
        iterate.last(itertools.islice(qvfs, num_episodes))
    
    list_of_qvfs.append(final_qvf)

In [68]:
from pprint import pprint

for t, q in enumerate(list_of_qvfs):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = max(
        ((q((NonTerminal(init_wealth), ac)), ac) for ac in alloc_choices),
        key=itemgetter(0)
    )[1]
    val: float = max(q((NonTerminal(init_wealth), ac))
                     for ac in alloc_choices)
    print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
    print("Optimal Weights below:")
    for wts in q.weights:
        pprint(wts.weights)
    print()

Time 0

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[4.80959858, 2.47903507, 3.82895144, 1.95879859]])

Time 1

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[2.95184702, 1.39973432, 2.71325493, 0.11627306]])

Time 2

Opt Risky Allocation = 2.000, Opt Val = -0.000
Optimal Weights below:
array([[2.33689038, 0.57447807, 2.19565247, 1.47547485]])

Time 3

Opt Risky Allocation = 1.000, Opt Val = -0.281
Optimal Weights below:
array([[-0.30656649,  1.27862432,  0.6007116 , -0.3043273 ]])



### Something still wrong for AssetAllocDiscrete implementation.... Not sure why